In [233]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [234]:
# Create engine using the `spacex.sqlite` database file
engine = create_engine("sqlite:///ufo.sqlite")

In [235]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

ufo
-----------
datetime DATETIME
city TEXT
state TEXT
country TEXT
shape TEXT
duration_seconds FLOAT
duration_hours_min TEXT
comments TEXT
date_posted TEXT
latitude FLOAT
longitude FLOAT
hour INTEGER
month INTEGER
year INTEGER
dayofweek INTEGER
day_of_week TEXT
category TEXT



In [237]:
# Genie query
# build the query
query = f"""
SELECT 
    state,
    shape,
    hour,
    month,
    year,
    day_of_week
FROM ufo
;
"""

# execute query
table_df = pd.read_sql(text(query), con=engine)
table_df.head(10)

,state,shape,hour,month,year,dayofweek
0,ny,rectangle,18,10,1994,0
1,ar,light,22,10,1994,0
2,wi,unknown,19,10,1995,1
3,ca,unknown,22,10,1995,1
4,mo,triangle,3,10,1996,3
5,sc,light,22,10,1996,3
6,oh,cylinder,22,10,1996,3
7,az,sphere,17,10,1997,4
8,ga,triangle,20,10,1997,4
9,mn,other,21,10,1997,4


In [238]:
# Genie query
table_df.columns = ['state','shape','hour','month','year','day_of_week']
print(table_df.groupby('dayofweek'))

In [239]:
# Genie query
# dataframe exploration

pivot1 = pd.pivot_table(table_df, values='state', index=['day_of_week'],
                       columns=['hour'], aggfunc=np.ma.count)
pivot1

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
dayofweek,,,,,,,,,,,,,,,,,,,,,
0,364,266,197,188,178,194,135,92,72,105,...,111,116,128,239,407,582,829,1023,948,650
1,377,321,210,190,138,202,152,115,90,100,...,121,113,152,276,465,694,864,1147,1004,683
2,410,303,176,197,152,207,130,103,81,105,...,111,121,152,272,392,650,939,1239,1127,741
3,330,283,227,177,151,164,153,98,78,101,...,96,140,159,267,419,669,936,1324,1046,751
4,406,313,234,173,152,177,146,97,106,118,...,107,138,150,265,387,661,940,1305,1236,860
5,475,400,269,233,170,178,115,108,93,127,...,170,169,187,315,504,756,1117,1640,1613,1130
6,529,372,266,232,205,156,112,55,83,86,...,157,182,204,267,425,624,920,1150,1104,798


In [240]:
# Genie query
pivot_df = pd.DataFrame(pd.pivot_table(table_df, values='state', index=['day_of_week'],
                       columns=['hour'], aggfunc=np.ma.count).sort_index(ascending=True))
pivot_df

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
dayofweek,,,,,,,,,,,,,,,,,,,,,
0,364,266,197,188,178,194,135,92,72,105,...,111,116,128,239,407,582,829,1023,948,650
1,377,321,210,190,138,202,152,115,90,100,...,121,113,152,276,465,694,864,1147,1004,683
2,410,303,176,197,152,207,130,103,81,105,...,111,121,152,272,392,650,939,1239,1127,741
3,330,283,227,177,151,164,153,98,78,101,...,96,140,159,267,419,669,936,1324,1046,751
4,406,313,234,173,152,177,146,97,106,118,...,107,138,150,265,387,661,940,1305,1236,860
5,475,400,269,233,170,178,115,108,93,127,...,170,169,187,315,504,756,1117,1640,1613,1130
6,529,372,266,232,205,156,112,55,83,86,...,157,182,204,267,425,624,920,1150,1104,798


In [241]:
# Genie query
import plotly.express as px
fig = px.imshow(pivot_df, 
                x=['12am','1am','2am','3am','4am','5am','6am','7am','8am','9am','10am','11am','12pm','1pm','2pm','3pm','4pm','5pm','6pm','7pm','8pm','9pm','10pm','11pm'], 
                y=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                color_continuous_scale='Greens', 
                text_auto=True,
                title="UFO Sightings Grouped by Time and Day of Week")
fig.update_layout(width=1000,height=500, font_family="Courier New", font_color="white", paper_bgcolor="black" )

fig.show()